In [17]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization
#CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint


In [3]:
df=pd.read_csv("crypto_data/LTC-USD.csv",names=["time","low","high","open","close","volume"])
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [4]:
main_df=pd.DataFrame()

In [5]:
ratios=["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]
for r in ratios:
    ds=f"crypto_data/{r}.csv"
    df=pd.read_csv(ds,names=["time","low","high","open","close","volume"])
    df.rename(columns={"close":f"{r}_close","volume":f"{r}_volume"},inplace=True)
    df.set_index("time",inplace=True)
    df=df[[f"{r}_close",f"{r}_volume"]]
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)
print(main_df.head())     
for i in main_df.columns:
    print(i)

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  
time                                                                      
1528968660            NaN             NaN     871.719971        5.675361  
1528968720      486.01001       26.019083     870.859985       26.856577  
1528968780      486.00000        8.449400     870.099976        1.124300  
1528968840      485.75000       26.994646     870.789978        1.749862  
1528968900      4

In [26]:
SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT="LTC-USD"
EPOCHS=10
BATCH_SIZE=64
NAME=f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
def classify(curr,future):
    if float(curr)<float(future):
        return 1
    else:
        return 0
 #scale and normalise   
def preprocess_df(df):
    df=df.drop("future",1)
    for c in df.columns:
        if c!="target":
            df[c]=df[c].pct_change()
            df.dropna(inplace=True)
            df[c]=preprocessing.scale(df[c].values)
    df.dropna(inplace=True)
    sequential_data=[]
    prev_days=deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days)==SEQ_LEN:
            sequential_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequential_data)      
    buys=[]
    sells=[]
    for seq,target in sequential_data:
        if target==0:
            sells.append([seq,target])
        elif target==1:
            buys.append([seq,target])
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower=min(len(buys),len(sells))
    buys=buys[:lower]
    sells=sells[:lower]
    sequential_data=buys+sells
    random.shuffle(sequential_data)
    
    x=[]
    y=[]
    for seq,target in sequential_data:
        x.append(seq)
        y.append(target)
    return np.array(x),y   

In [7]:
main_df['future']=main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future
time,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001


In [8]:
print(main_df[[f"{RATIO_TO_PREDICT}_close","future"]])

            LTC-USD_close     future
time                                
1528968660      96.580002  96.500000
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997
1528968840      96.500000  96.440002
1528968900      96.389999  96.470001
...                   ...        ...
1535214960      58.020000  58.020000
1535215020      58.009998  58.080002
1535215080      58.020000        NaN
1535215140      58.020000        NaN
1535215200      58.080002        NaN

[97724 rows x 2 columns]


In [9]:
main_df["target"]=list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df["future"]))

In [10]:
print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]])

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
...                   ...        ...     ...
1535214960      58.020000  58.020000       0
1535215020      58.009998  58.080002       1
1535215080      58.020000        NaN       0
1535215140      58.020000        NaN       0
1535215200      58.080002        NaN       0

[97724 rows x 3 columns]


In [11]:
times=sorted(main_df.index.values)

In [12]:
last_5pct=times[-int(0.05*len(times))]

In [13]:
print(last_5pct)

1534922100


In [14]:
validation_main_df=main_df[(main_df.index>=last_5pct)]
main_df=main_df[(main_df.index<last_5pct)]

In [34]:
#preprocess_df(main_df)
train_x,train_y=preprocess_df(main_df)
v_x,v_y=preprocess_df(validation_main_df)

print(type(train_x))

<class 'numpy.ndarray'>


In [35]:
print(f"train data:{len(train_x)} validation data:{len(v_x)} ")
print(f"Not to buy: {train_y.count(0)}, To buy: {train_y.count(1)}")
print(f"Validation :Not to buy: {v_y.count(0)}, To buy: {v_y.count(1)}")      

train data:69188 validation data:3062 
Not to buy: 34594, To buy: 34594
Validation :Not to buy: 1531, To buy: 1531


In [49]:
model=Sequential()
model.add(LSTM(128,input_shape=(train_x.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model=Sequential()
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model=Sequential()
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2,activation='softmax'))

opt=tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

tensorboard=TensorBoard(log_dir=f'logs/{NAME}')
file_path= "RNN_Final-{epoch:02d}"
checkpoint=ModelCheckpoint("models/{}.model".format(file_path,monitor='val_acc',verbose=1,save_best_only=True,mode='max'))


train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
v_x = np.asarray(v_x)
v_y = np.asarray(v_y)

history=model.fit(train_x,train_y,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_data=(v_x,v_y),callbacks=[tensorboard,checkpoint])



Epoch 1/10
1082/1082 [==============================] - 56s 50ms/step - loss: 0.7325 - accuracy: 0.5261 - val_loss: 0.6845 - val_accuracy: 0.5624


INFO:tensorflow:Assets written to: models\RNN_Final-01.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-01.model\assets


Epoch 2/10
1082/1082 [==============================] - 80s 74ms/step - loss: 0.6865 - accuracy: 0.5482 - val_loss: 0.6759 - val_accuracy: 0.5754


INFO:tensorflow:Assets written to: models\RNN_Final-02.model\assets


...

INFO:tensorflow:Assets written to: models\RNN_Final-09.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-09.model\assets


Epoch 10/10
1082/1082 [==============================] - 91s 84ms/step - loss: 0.6743 - accuracy: 0.5774 - val_loss: 0.6829 - val_accuracy: 0.5594


INFO:tensorflow:Assets written to: models\RNN_Final-10.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-10.model\assets


In [50]:
score = model.evaluate(v_x, v_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

Test loss: 0.6828904151916504
Test accuracy: 0.5594382882118225


INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1612085417\assets


INFO:tensorflow:Assets written to: models/60-SEQ-3-PRED-1612085417\assets
